## Task Description
Mental Illness Detection with BERTopic



#### Install Dependencies and Restart Runtime

In [ ]:
!pip install -q transformers
!pip install -q simpletransformers`
!pip install -U imbalanced-learn`

/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file
/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/'CS 7650'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CS 7650


# Getting Data

In [ ]:
## Importing packages
import pandas as pd
import numpy as np
import requests
import json
import re

#Feching and storing the data

df = pd.read_excel('reddit_data.xlsx')

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 24,057



,ID,Title,Text,Date,Label,Is_MH
17249,d5katw,whenever i am in a public i always think i see...,Whenever i go out and walk past someone not ev...,2019-09-17 17:34:49,ptsd,Yes
16284,b0emw8,Spring is in the air.,NaN,2019-03-12 23:17:23,bipolar,Yes
13625,1fvh38z,How do I even ask for help,"My family is so judgemental over everything, t...",2024-10-03 20:14:54,depression,Yes
14704,508b00,"Could this be schizophrenia, or am I just para...",Last year I began experiencing some symptoms w...,2016-08-30 00:27:14,schizophrenia,Yes
703,13uhwsx,Why is it so hard to M********te?(NSFW),Hi. I(f18) have a really hard time taking my h...,2023-05-29 02:06:10,ptsd,Yes
5168,1fgf72,My PTSD has consumed my life,At the age of 16 I was in a horrible car accid...,2013-06-01 08:07:30,ptsd,Yes
9249,1fsmqfw,will i ever get another job,about 2 months ago i worked at a sandwich shop...,2024-09-30 03:50:16,anxiety,Yes
18776,gl0i34,Just got an email from my school district sayi...,My school district just sent out an email toni...,2020-05-16 19:16:47,anxiety,Yes
1283,17bbfmk,Does anyone get the anxiety poops?,"Totally gross and TMI, but I’ve noticed since ...",2023-10-19 05:17:35,anxiety,Yes
11952,1fughc7,How can psych ward help lol,Ok assume that I tell someone that I am active...,2024-10-02 13:42:02,depression,Yes


In [ ]:
df['Title'] = df['Title'].fillna('')
df['Text'] = df['Text'].fillna('')
df['Full_Text'] = df['Title'] + ' ' + df['Text']
df['Full_Text'] = df['Full_Text'].astype(str)

In [ ]:
df.head()

,ID,Title,Text,Date,Label,Is_MH,Full_Text
0,100jk9m,The Person Who Traumatized You DMed you on Ins...,The title says it all. I came back home from a...,2023-01-01 13:21:11,ptsd,Yes,The Person Who Traumatized You DMed you on Ins...
1,100t81n,PTSD from antibiotics,I took an antibiotic called ciprofloxacin a fe...,2023-01-01 20:57:25,ptsd,Yes,PTSD from antibiotics I took an antibiotic cal...
2,100xfwl,guys i made a phone call!!!!!!!!,basically my credit card was stolen and loads ...,2023-01-01 23:54:55,anxiety,Yes,guys i made a phone call!!!!!!!! basically my ...
3,100y8dh,The Time when I was Sexually Harassed online (...,*from a user i talked to who doesnt use Reddit...,2023-01-02 00:29:48,ptsd,Yes,The Time when I was Sexually Harassed online (...
4,101kgaa,Having a panic attack due to real event. need ...,TW: POCD. 22F. So I have severe OCD and on med...,2023-01-02 18:59:21,OCD,Yes,Having a panic attack due to real event. need ...


In [ ]:
df_subset1 = df[['ID', 'Full_Text','Label']]
df_subset1 = df_subset1.rename(columns={'Full_Text' : 'text'})
label_mapping = {'none': 0, 'anxiety': 1, 'depression': 2, 'OCD': 3, 'ptsd': 4, 'schizophrenia': 5, 'bipolar': 6, 'anorexia': 7}
df_subset1['labels'] = df_subset1['Label'].map(label_mapping)
df_subset1 = df_subset1.drop(['Label'], axis=1)
df_subset1.head()

,ID,text,labels
0,100jk9m,The Person Who Traumatized You DMed you on Ins...,4
1,100t81n,PTSD from antibiotics I took an antibiotic cal...,4
2,100xfwl,guys i made a phone call!!!!!!!! basically my ...,1
3,100y8dh,The Time when I was Sexually Harassed online (...,4
4,101kgaa,Having a panic attack due to real event. need ...,3


In [ ]:
#Create a function to clean text in the datafrmae
def clean_text(text):

  # Remove emojis
  text = re.sub(r'[:;][\w\d-]*[:;]', '', text)

  # Remove emoticons
  text = re.sub(r'\([^\)]*\)', '', text)

  # Remove hyperlinks
  text = re.sub(r'https?://[^\s]+', '', text)

  # Remove non-English characters
  text = re.sub(r'[^\w\s]', '', text)

  # Convert all text into lowercase
  text = text.lower()

  return text


In [ ]:
df_subset1['text'] = df_subset1['text'].apply(clean_text)
df_subset1.sample(10)

,ID,text,labels
15853,a2a4u7,anyone else get anxiety when you notice the sl...,1
9733,1ft0aex,is this program too intense so its 426h00 in ...,0
9579,1fswf68,a tour of ancient cities in turkey if you are ...,0
2415,1cq1cfh,i have learnt recently that an infant baby to...,4
553,133vye,to those taking online tests to ascertain if y...,3
23542,wpa3y4,uhh please read for a while now ive been deali...,5
15412,8orsvo,i am 58 m seems to be most subscribers are you...,2
47,107xsox,can i get a woot woot for 10 months,6
14938,6kfdyy,i really dont want to die i actually really li...,2
3860,1f2z4ls,sometimes i think that i would be better off d...,2


In [ ]:
%%capture
!pip install bertopic
from umap import UMAP
umap_model = UMAP(random_state=123)

In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, umap_model=umap_model)
topic_df = df_subset1.copy()
topic_df.head()

,ID,text,labels
0,100jk9m,the person who traumatized you dmed you on ins...,4
1,100t81n,ptsd from antibiotics i took an antibiotic cal...,4
2,100xfwl,guys i made a phone call basically my credit c...,1
3,100y8dh,the time when i was sexually harassed online ...,4
4,101kgaa,having a panic attack due to real event need r...,3


In [ ]:
topics, probs = topic_model.fit_transform(topic_df['text'])
freq = topic_model.get_topic_info()
freq.head(5)

2024-11-16 20:19:02,777 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/752 [00:00<?, ?it/s]

2024-11-16 20:20:19,391 - BERTopic - Embedding - Completed ✓
2024-11-16 20:20:19,392 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-16 20:20:42,123 - BERTopic - Dimensionality - Completed ✓
2024-11-16 20:20:42,127 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-16 20:22:09,275 - BERTopic - Cluster - Completed ✓
2024-11-16 20:22:09,287 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-16 20:22:11,716 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,8625,-1_to_and_my_me,"[to, and, my, me, the, of, that, it, in, im]",[please downvote i know ive no right to feel i...
1,0,2489,0_eat_weight_eating_anorexia,"[eat, weight, eating, anorexia, food, ed, reco...",[how chronic anorexia feels it feels painful\...
2,1,924,1_schizophrenia_voices_hallucinations_schizoph...,"[schizophrenia, voices, hallucinations, schizo...",[i think i have schizophrenia but i dont want ...
3,2,811,2_ocd_thoughts_you_your,"[ocd, thoughts, you, your, it, is, that, this,...","[ocd be like , ocd be like , wisdom the prize ..."
4,3,744,3_bipolar_manic_mania_episode,"[bipolar, manic, mania, episode, meds, disorde...",[might be bipolar so the point of this post is...


In [ ]:
dict_topic = pd.Series(freq.Name.values,index=freq.Topic).to_dict()
topic_df['topics'] = topics
topic_df['topic_label'] = topic_df['topics'].replace(dict_topic)
topic_df[0:10]

,ID,text,labels,topics,topic_label
0,100jk9m,the person who traumatized you dmed you on ins...,4,54,54_um_patient_like_but
1,100t81n,ptsd from antibiotics i took an antibiotic cal...,4,14,14_heart_anxiety_chest_attack
2,100xfwl,guys i made a phone call basically my credit c...,1,69,69_im_friends_am_depressed
3,100y8dh,the time when i was sexually harassed online ...,4,-1,-1_to_and_my_me
4,101kgaa,having a panic attack due to real event need r...,3,-1,-1_to_and_my_me
5,101o0qc,off topic whose your favorite music artist im ...,7,59,59_mask_masks_songs_wear
6,101pxbs,how tf am i the weight i used to be in jr high...,7,0,0_eat_weight_eating_anorexia
7,101z2y0,do you find this game i found at barnes noble...,6,7,7_writing_story_write_book
8,101zp62,little poem about mental health,6,-1,-1_to_and_my_me
9,1022fl4,jock itchcrotch rot sorry if this is tmi and d...,2,52,52_rabies_pee_bladder_bat


In [ ]:
topic_df['text'] = topic_df['topic_label'].map(str) + ': ' + topic_df['text'].map(str)
topic_df = topic_df.drop(['topics', 'topic_label'], axis =1)
topic_df.head()

,ID,text,labels
0,100jk9m,54_um_patient_like_but: the person who traumat...,4
1,100t81n,14_heart_anxiety_chest_attack: ptsd from antib...,4
2,100xfwl,69_im_friends_am_depressed: guys i made a phon...,1
3,100y8dh,-1_to_and_my_me: the time when i was sexually ...,4
4,101kgaa,-1_to_and_my_me: having a panic attack due to ...,3


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(topic_df, test_size=0.20, random_state=123)

In [ ]:
train_df.head()

,ID,text,labels
15374,8jm8ia,1_schizophrenia_voices_hallucinations_schizoph...,5
180,10xcw1t,0_eat_weight_eating_anorexia: how do i help so...,7
21835,ofepru,-1_to_and_my_me: contamination from pollutants...,3
11200,1fu00z6,63_anxiety_anxious_your_books: are books work...,1
21169,ma4btf,-1_to_and_my_me: just fucking hug me i sit her...,1


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19245 entries, 15374 to 19966
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      19245 non-null  object
 1   text    19245 non-null  object
 2   labels  19245 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 601.4+ KB


In [ ]:
pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00
  

In [ ]:
print(train_df.dtypes)

ID        object
text      object
labels     int64
dtype: object


# Training and Testing the Model

In [ ]:
import os
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 256,
    'num_train_epochs': 3,
    'train_batch_size': 16,
    'fp16': True,
    'output_dir': '/outputs/',
}

In [ ]:
#RoBERTa
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel('roberta', 'roberta-base', num_labels=8, args=train_args)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Train the model, there is no development or validation set for this dataset
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)
print(result)


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Running Epoch 3 of 3:   0%|          | 0/1589 [00:00<?, ?it/s]

  0%|          | 0/4812 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (820 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (540 > 512). Running this sequence through the model will result in indexing errors


Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

{'mcc': 0.7636105806672076, 'acc': 0.792809642560266, 'eval_loss': 0.723072716167995}


In [ ]:
#F1 Score
from sklearn.metrics import f1_score

def f1_macro(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=f1_macro)
print(result)

  0%|          | 0/4812 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (820 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (540 > 512). Running this sequence through the model will result in indexing errors


Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

{'mcc': 0.7636105806672076, 'acc': 0.790668164748832, 'eval_loss': 0.723072716167995}


In [ ]:
#Precision and Recall
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.precision_recall_fscore_support)
print(result)

  0%|          | 0/4812 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (820 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (540 > 512). Running this sequence through the model will result in indexing errors


Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

{'mcc': 0.7636105806672076, 'acc': (array([0.95988935, 0.62774725, 0.7712305 , 0.76491863, 0.89753321,
       0.74377224, 0.68533773, 0.91401869]), array([0.95460798, 0.80175439, 0.79322638, 0.72931034, 0.80442177,
       0.71088435, 0.65822785, 0.8639576 ]), array([0.95724138, 0.70416025, 0.78207381, 0.7466902 , 0.84843049,
       0.72695652, 0.67150928, 0.88828338]), array([727, 570, 561, 580, 588, 588, 632, 566])), 'eval_loss': 0.723072716167995}


# RoBERTa


In [ ]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 256,
    'num_train_epochs': 3,
    'train_batch_size': 16,
    'fp16': True,
    'output_dir': '/outputs/',
}

In [ ]:
#BERT
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

model2 = ClassificationModel('bert', 'bert-base-uncased', num_labels=8, args=train_args)

# Train the model, there is no development or validation set for this dataset
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model2.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model2.eval_model(test_df, acc=sklearn.metrics.accuracy_score)
print(result)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/19245 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (887 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/1571 [00:00<?, ?it/s]

In [ ]:
#F1 Score
from sklearn.metrics import f1_score

def f1_macro(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

result, model_outputs, wrong_predictions = model2.eval_model(test_df, acc=f1_macro)
print(result)

In [ ]:
 #Precision and Recall
result, model_outputs, wrong_predictions = model2.eval_model(test_df, acc=sklearn.metrics.precision_recall_fscore_support)
print(result)